In [17]:
!pip install interface_meta --no-index --find-links=file:../input/e2eml-inc-dependencies/interface_meta-1.2.4-py2.py3-none-any.whl
!pip install astor --no-index --find-links=file:../input/e2eml-inc-dependencies/astor-0.8.1-py2.py3-none-any.whl
!pip install formulaic --no-index --find-links=file:../input/formulaic/formulaic-0.2.4-py3-none-any.whl
!pip install autograd --no-index --find-links=file:../input/e2eml-inc-dependencies/autograd-1.3-py3-none-any.whl
!pip install autograd-gamma --no-index --find-links=file:../input/e2eml-inc-dependencies/autograd-gamma-0.5.0/dist/autograd-gamma-0.5.0.tar
!pip install lifelines --no-index --find-links=file:../input/lifelines/lifelines-0.26.4-py3-none-any.whl
!pip install ngboost --no-index --find-links=file:../input/e2eml-inc-dependencies/ngboost-0.3.12-py3-none-any.whl

Looking in links: file:///../input/e2eml-inc-dependencies/interface_meta-1.2.4-py2.py3-none-any.whl
Looking in links: file:///../input/e2eml-inc-dependencies/astor-0.8.1-py2.py3-none-any.whl
Looking in links: file:///../input/formulaic/formulaic-0.2.4-py3-none-any.whl
Looking in links: file:///../input/e2eml-inc-dependencies/autograd-1.3-py3-none-any.whl
Looking in links: file:///../input/e2eml-inc-dependencies/autograd-gamma-0.5.0/dist/autograd-gamma-0.5.0.tar
Looking in links: file:///../input/lifelines/lifelines-0.26.4-py3-none-any.whl
Looking in links: file:///../input/e2eml-inc-dependencies/ngboost-0.3.12-py3-none-any.whl


In [18]:
!pip install boostaroota --no-index --find-links=file:../input/e2eml-inc-dependencies/boostaroota-1.3-py2.py3-none-any.whl
!pip install matplotlib --no-index --find-links=file:../input/e2eml-inc-dependencies/matplotlib-3.1.3-cp38-cp38-manylinux1_x86_64.whl

Looking in links: file:///../input/e2eml-inc-dependencies/boostaroota-1.3-py2.py3-none-any.whl
Looking in links: file:///../input/e2eml-inc-dependencies/matplotlib-3.1.3-cp38-cp38-manylinux1_x86_64.whl


In [19]:
!pip install catboost --no-index --find-links=file:../input/e2eml-inc-dependencies/catboost-0.21-cp38-none-manylinux1_x86_64.whl

Looking in links: file:///../input/e2eml-inc-dependencies/catboost-0.21-cp38-none-manylinux1_x86_64.whl


In [20]:
!pip install pytorch_tabnet --no-index --find-links=file:../input/e2eml-inc-dependencies/pytorch_tabnet-3.1.1-py3-none-any.whl

Looking in links: file:///../input/e2eml-inc-dependencies/pytorch_tabnet-3.1.1-py3-none-any.whl


In [21]:
!pip install shap --no-index --find-links=file:../input/e2eml-inc-dependencies/shap-0.39.0-cp35-cp35m-linux_armv6l.whl

Looking in links: file:///../input/e2eml-inc-dependencies/shap-0.39.0-cp35-cp35m-linux_armv6l.whl


In [22]:
!pip install e2eml --no-index --no-dependencies --find-links=file:../input/e2eml-inc-dependencies/e2eml-2.10.4-py3-none-any.whl

Looking in links: file:///../input/e2eml-inc-dependencies/e2eml-2.10.4-py3-none-any.whl


In [23]:
!TRANSFORMERS_OFFLINE=1

In [24]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
# load libraries
from e2eml.regression import regression_blueprints
from e2eml.full_processing import postprocessing
from e2eml.full_processing.postprocessing import save_to_production, load_for_production
import gc

In [25]:
target = "score"

In [26]:
df_train = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
df_train = df_train.rename(columns={"text": "comment_text"})

In [27]:
toxic_ml = load_for_production(file_path='../input/toxic-bert-auto-ml/', file_name='toxic_ml_bert_instance.dat')
toxic_ml.transformer_chosen='../input/toxic-bert-auto-ml'
toxic_ml.transformer_model_save_states_path='../input/toxic-bert-auto-ml'
toxic_ml.transformer_settings["pred_batch_size"] = 1
toxic_ml.ml_bp16_regressions_full_processing_bert_transformer(df_train)
df_train["bert_score"] = toxic_ml.predicted_values["nlp_transformer"]

Started Execute test train split at 18:33:50.
Started Handle rare features at 18:33:50.
Started Sort columns alphabetically at 18:33:50.


Some weights of the model checkpoint at ../input/toxic-bert-auto-ml were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ../input/toxic-bert-auto-ml and are newly initialized: ['classifier.out_proj.weight', 'classifier.d

['../input/toxic-bert-auto-ml/model1.pth', '../input/toxic-bert-auto-ml/model2.pth', '../input/toxic-bert-auto-ml/model4.pth']


In [28]:
batch_size = 2500
max_len = len(df_train.index)

if max_len%batch_size == 0:
    nb_batches = int(max_len/batch_size)
else:
    nb_batches = int(max_len/batch_size + 1)

In [29]:
results = []
already_batched = 0

for batch in range(nb_batches):
    start_idx = already_batched
    end_idx = already_batched + batch_size
    if max_len - already_batched < batch_size:
        batch_size = max_len-already_batched
        end_idx = already_batched + batch_size
    
    temp_df = df_train[["comment_text"]].iloc[start_idx:end_idx, :].copy()
    toxic_ml = load_for_production(file_path='../input/toxic-auto-ml/', file_name='toxic_ml_instance.dat')
    toxic_ml.blueprint_step_selection_non_nlp["trained_tokenizer_embedding"] = False
    toxic_ml.ml_bp20_regression_full_processing_sgd(temp_df)
    val_y_hat = toxic_ml.predicted_values['sgd']
    results.append(pd.Series(val_y_hat))
    already_batched += batch_size
    del temp_df
    del toxic_ml
    del val_y_hat
    _ = gc.collect()

Started Execute test train split at 18:41:31.
Started Started column type detection and casting at 18:41:31.
Started Checking for duplicate columns at 18:41:31.
Started Reset dataframe index. at 18:41:31.
Started Delete columns with high share of NULLs at 18:41:31.
Started Start text cleaning. at 18:41:31.
Started Start TFIDF to PCA loop at 18:41:34.
Started PCA POS tags at 18:41:35.
Started Start TFIDF to PCA loop at 18:41:36.
Started Holistic NULL filling at 18:41:38.
Started Execute categorical encoding at 18:41:38.
Started Fill nulls at 18:41:39.
Started Predict with SGD regression at 18:41:44.
Started Target skewness handling at 18:41:44.
Started Execute test train split at 18:41:45.
Started Started column type detection and casting at 18:41:45.
Started Checking for duplicate columns at 18:41:45.
Started Reset dataframe index. at 18:41:45.
Started Delete columns with high share of NULLs at 18:41:45.
Started Start text cleaning. at 18:41:45.
Started Start TFIDF to PCA loop at 18:41

In [30]:
res_ser = pd.concat(results)
df_train["sgd_score"] = res_ser.values

In [31]:
df_train["score"] = (df_train["sgd_score"]+df_train["bert_score"])/2
df_train = df_train[['comment_id','score']]
df_train.to_csv('./submission.csv', index=False)
df_train

,comment_id,score
0,114890,0.284119
1,732895,0.445234
2,1139051,0.284343
3,1434512,0.308836
4,2084821,0.565980
...,...,...
7532,504235362,0.701044
7533,504235566,0.522043
7534,504308177,0.311612
7535,504570375,0.647681
